
# Русский стемминг и лемматизация — примеры


- **Стемминг** (Snowball, NLTK)
- **Лемматизация**: **pymorphy2** и **pymystem3**

> Если пакет не установлен, раскомментируйте `pip install` в соответствующей ячейке.


## 1. Примерные слова и фразы

In [ ]:

WORDS = [
    "машины", "машинке", "поездки", "поехать", "бегаю", "бегала",
    "хорошие", "лучший", "прочитанная", "прочитать", "синие", "синего",
    "детям", "детский", "идущего", "идти", "вкуснейший", "вкусный"
]

TEXTS = [
    "Мне не понравились машины: слишком дорогие и неудобные.",
    "Книга прочитанная мной — лучшая за этот год.",
    "Детский лагерь оказался очень уютным и дружелюбным.",
]
print("Пример слов:", WORDS[:8])
print("Пример фраз:", TEXTS[0])


Пример слов: ['машины', 'машинке', 'поездки', 'поехать', 'бегаю', 'бегала', 'хорошие', 'лучший']
Пример фраз: Мне не понравились машины: слишком дорогие и неудобные.


## 2. Стемминг (NLTK SnowballStemmer)

In [ ]:

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
stems = [stemmer.stem(w.lower()) for w in WORDS]
for w, s in zip(WORDS, stems):
    print(f"{w:12s} -> {s}")


машины       -> машин
машинке      -> машинк
поездки      -> поездк
поехать      -> поеха
бегаю        -> бега
бегала       -> бега
хорошие      -> хорош
лучший       -> лучш
прочитанная  -> прочита
прочитать    -> прочита
синие        -> син
синего       -> син
детям        -> дет
детский      -> детск
идущего      -> идущ
идти         -> идт
вкуснейший   -> вкусн
вкусный      -> вкусн


## 3. Лемматизация (pymorphy2)

In [ ]:
# !pip -q install pymorphy2

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
print(morph.parse("детям"))
print(morph.parse("детям")[0])
print(morph.parse("детям")[0].normal_form)
print(morph.parse("детям")[0].tag.POS)

#see https://pymorphy2.readthedocs.io/en/stable/user/guide.html

[Parse(word='детям', tag=OpencorporaTag('NOUN,anim,masc plur,datv'), normal_form='ребёнок', score=1.0, methods_stack=((DictionaryAnalyzer(), 'детям', 2780, 8),))]
Parse(word='детям', tag=OpencorporaTag('NOUN,anim,masc plur,datv'), normal_form='ребёнок', score=1.0, methods_stack=((DictionaryAnalyzer(), 'детям', 2780, 8),))
ребёнок
NOUN


In [ ]:

lemmas_pymorphy = [morph.parse(w)[0].normal_form for w in WORDS]
print("pymorphy2 леммы:")
for w, l in zip(WORDS, lemmas_pymorphy):
    print(f"{w:12s} -> {l}")



pymorphy2 леммы:
машины       -> машина
машинке      -> машинка
поездки      -> поездка
поехать      -> поехать
бегаю        -> бегать
бегала       -> бегать
хорошие      -> хороший
лучший       -> хороший
прочитанная  -> прочитать
прочитать    -> прочитать
синие        -> синий
синего       -> синий
детям        -> ребёнок
детский      -> детский
идущего      -> идти
идти         -> идти
вкуснейший   -> вкусный
вкусный      -> вкусный


## 4. Лемматизация (pymystem3 / Yandex Mystem)

In [ ]:

# !pip -q install pymystem3
try:
    from pymystem3 import Mystem
    mystem = Mystem()
    lemmas_mystem_words = [mystem.lemmatize(w)[0] for w in WORDS]
    lemmas_mystem_texts = [" ".join([t for t in mystem.lemmatize(s) if t.strip()]) for s in TEXTS]

    print("pymystem3 леммы (слова):")
    for w, l in zip(WORDS, lemmas_mystem_words):
        print(f"{w:12s} -> {l.strip()}")

    print("\nЛемматизация фраз (pymystem3):")
    for s, l in zip(TEXTS, lemmas_mystem_texts):
        print("Исходное:", s)
        print("Леммы:   ", l.strip())
        print("-" * 40)
except Exception as e:
    print("pymystem3 недоступен. Установите пакет и повторите. Ошибка:", e)


pymystem3 леммы (слова):
машины       -> машина
машинке      -> машинка
поездки      -> поездка
поехать      -> поехать
бегаю        -> бегать
бегала       -> бегать
хорошие      -> хороший
лучший       -> хороший
прочитанная  -> прочитывать
прочитать    -> прочитывать
синие        -> синий
синего       -> синий
детям        -> ребенок
детский      -> детский
идущего      -> идти
идти         -> идти
вкуснейший   -> вкусный
вкусный      -> вкусный

Лемматизация фраз (pymystem3):
Исходное: Мне не понравились машины: слишком дорогие и неудобные.
Леммы:    я не понравиться машина :  слишком дорогой и неудобный .
----------------------------------------
Исходное: Книга прочитанная мной — лучшая за этот год.
Леммы:    книга прочитывать я  —  хороший за этот год .
----------------------------------------
Исходное: Детский лагерь оказался очень уютным и дружелюбным.
Леммы:    детский лагерь оказываться очень уютный и дружелюбный .
----------------------------------------


## 5. Обработка «не» + лемматизация

In [ ]:

import re
token_re = re.compile(r"[А-Яа-яЁёA-Za-z]+")

def get_lem_fn():
    m = pymorphy2.MorphAnalyzer()
    return lambda tok: m.parse(tok)[0].normal_form


def lemmatize(text):
    m = pymorphy2.MorphAnalyzer()
    tokens = token_re.findall(text.lower())
    out = []
    for t in tokens:
        if t == "не":
            out.append(t)
        else:
            out.append( m.parse(t)[0].normal_form)
    return out

for s in TEXTS:
    print("Текст: ", s)
    print("Леммы с 'не':", lemmatize(s))
    print("-"*40)


Текст:  Мне не понравились машины: слишком дорогие и неудобные.
Леммы с 'не': ['я', 'не', 'понравиться', 'машина', 'слишком', 'дорогой', 'и', 'неудобный']
----------------------------------------
Текст:  Книга прочитанная мной — лучшая за этот год.
Леммы с 'не': ['книга', 'прочитать', 'я', 'хороший', 'за', 'этот', 'год']
----------------------------------------
Текст:  Детский лагерь оказался очень уютным и дружелюбным.
Леммы с 'не': ['детский', 'лагерь', 'оказаться', 'очень', 'уютный', 'и', 'дружелюбный']
----------------------------------------


## 6. Сравнение: стемминг vs лемматизация

In [ ]:

def compare_stem_lemma(words):
    from nltk.stem.snowball import SnowballStemmer
    st = SnowballStemmer("russian")
    stems = [st.stem(w.lower()) for w in words]

    lemma_fn = get_lem_fn()
    lemmas = [lemma_fn(w.lower()) for w in words]

    print(f"{'Слово':14s} | {'Стим':12s} | {'Лемма':12s}")
    print("-"*44)
    for w, s, l in zip(words, stems, lemmas):
        print(f"{w:14s} | {s:12s} | {l:12s}")

compare_stem_lemma(WORDS[:12])


Слово          | Стим         | Лемма       
--------------------------------------------
машины         | машин        | машина      
машинке        | машинк       | машинка     
поездки        | поездк       | поездка     
поехать        | поеха        | поехать     
бегаю          | бега         | бегать      
бегала         | бега         | бегать      
хорошие        | хорош        | хороший     
лучший         | лучш         | хороший     
прочитанная    | прочита      | прочитать   
прочитать      | прочита      | прочитать   
синие          | син          | синий       
синего         | син          | синий       


## 7. Нормализация токенов

In [ ]:

def normalize_tokens(text, mode='lemma'):
    import re
    token_re = re.compile(r"[А-Яа-яЁёA-Za-z]+")
    toks = [t.lower() for t in token_re.findall(text)]
    if mode == 'raw':
        return toks
    elif mode == 'stem':
        from nltk.stem.snowball import SnowballStemmer
        st = SnowballStemmer("russian")
        return [st.stem(t) for t in toks]
    else:
        lemma_fn = get_lem_fn()
        return [lemma_fn(t) for t in toks]

sample = "Потрясающие машины стояли вдоль дороги, а дети радостно бегали."
for mode in ['raw', 'stem', 'lemma']:
    print(mode, "->", normalize_tokens(sample, mode=mode))


raw -> ['потрясающие', 'машины', 'стояли', 'вдоль', 'дороги', 'а', 'дети', 'радостно', 'бегали']
stem -> ['потряса', 'машин', 'стоя', 'вдол', 'дорог', 'а', 'дет', 'радостн', 'бега']
lemma -> ['потрясать', 'машина', 'стоять', 'вдоль', 'дорога', 'а', 'ребёнок', 'радостно', 'бегать']



## 8. Представления текста: Bag of Words (BoW) и TF-IDF

Перед тем как применять модели машинного обучения, текст нужно преобразовать в **числовой формат**.  
Существуют два базовых способа — **BoW** и **TF-IDF**.

---

### 1. Bag of Words (мешок слов)

**Идея:** представляем текст как набор слов без учёта их порядка.  
Для каждого слова из словаря (всех слов корпуса) считаем, сколько раз оно встречается в документе.

Пример:

| Документ | Текст |
|-----------|-------|
| D1 | "кошки любят молоко" |
| D2 | "собаки тоже любят молоко" |

Словарь (все уникальные слова):  
`["кошки", "любят", "молоко", "собаки", "тоже"]`

Тогда векторы:

| Слово      | D1 | D2 |
|-------------|----|----|
| кошки       | 1  | 0  |
| любят       | 1  | 1  |
| молоко      | 1  | 1  |
| собаки      | 0  | 1  |
| тоже        | 0  | 1  |

BoW-вектор для D1 = `[1, 1, 1, 0, 0]`

**Минус:** модель не различает порядок слов и значимость каждого слова — часто встречающиеся вроде «и», «в», «на» могут доминировать.

---

### 2. TF-IDF (Term Frequency — Inverse Document Frequency)

TF-IDF учитывает **вес слова**, то есть его важность в контексте всего корпуса.

**Формулы** для слова *t* в документе *d*:

\[
TF(t, d) = \frac{\text{кол-во вхождений t в d}}{\text{общее число слов в d}}
\]

\[
IDF(t) = \log\frac{N}{1 + n_t}
\]

где *N* — число документов, *nₜ* — в скольких документах встречается слово *t*.

\[
TFIDF(t, d) = TF(t, d) \times IDF(t)
\]

Чем чаще слово встречается в конкретном документе, но реже — во всём корпусе, тем выше его вес.  
TF-IDF подавляет «общие» слова (вроде “и”, “на”) и усиливает уникальные (“молоко”, “кошки”).

---

### 3. Интуитивно

| Слово | Частое везде | Редкое, но информативное |
|--------|---------------|--------------------------|
| и, но, на | низкий TF-IDF | — |
| кошка, молоко | — | высокий TF-IDF |

---

###  Вывод

- **BoW** — простая частотная модель (учёт словаря, подсчёт частот).  
- **TF-IDF** — «умная» версия BoW, добавляющая взвешивание по важности.  
Используется почти во всех классических NLP-моделях (логистическая регрессия, наивный Байес, SVM и др.).


In [ ]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

docs = [
    "кошки любят молоко",
    "собаки тоже любят молоко",
    "кошки не любят собак"
]

# --- BoW ---
bow = CountVectorizer()
X_bow = bow.fit_transform(docs)
bow_df = pd.DataFrame(X_bow.toarray(), columns=bow.get_feature_names_out())
print("Bag of Words (BoW):")
display(bow_df)

# --- TF-IDF ---
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(docs)
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
print("TF-IDF:")
display(tfidf_df.round(3))


Bag of Words (BoW):


,кошки,любят,молоко,не,собак,собаки,тоже
0,1,1,1,0,0,0,0
1,0,1,1,0,0,1,1
2,1,1,0,1,1,0,0


TF-IDF:


,кошки,любят,молоко,не,собак,собаки,тоже
0,0.620,0.481,0.620,0.000,0.000,0.000,0.000
1,0.000,0.345,0.445,0.000,0.000,0.584,0.584
2,0.445,0.345,0.000,0.584,0.584,0.000,0.000
